In [17]:
import numpy as np
from astropy.table import Table, vstack, join, unique

from astropy.io import fits
from astropy.coordinates import SkyCoord, match_coordinates_sky, search_around_sky
from astropy import units as u
from tqdm import tqdm

import os
import psycopg2
from psycopg2.extras import execute_values
from psycopg2.extensions import register_adapter, AsIs


import datetime

from desitarget.targets import decode_targetid, encode_targetid, resolve
from desitarget.io import releasedict, release_to_photsys


import requests
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle


In [2]:
def get_rot_curve_targets(redux, sga_params, verbose=False):
    """Get rotation curve targets from the DESI production DB for a given spectroscopic reduction.
    
    Parameters
    ----------
    redux : str
        Spectroscopic reduction. can be 'fuji', 'guadalupe', 'iron', or 'loa'
    ra : float
        galaxy RA from SGA [deg]
    dec : float
        galaxy dec from SGA [deg]
    major : float
        semi-major axis (D26/2) from SGA [deg]
    ratio : float
        axis ratio (BA) from SGA
    pa : float
        position angle east-of-north from SGA [deg]
    
    Returns
    -------
    targets : Table
        Table of observations.
    """
    
    rot_curve_targets = Table(names=['TARGETID', 'TILEID', 'TARGET_RA', 'TARGET_DEC', 'HEALPIX', 'SURVEY', 'PROGRAM',
                          'Z', 'ZERR', 'ZWARN', 'CHI2', 'DELTACHI2', 'MEAN_FIBER_RA', 'MEAN_FIBER_DEC',
                         'STD_FIBER_RA', 'STD_FIBER_DEC', 'SPECTYPE', 'SGA_ID'],
          dtype=['int64', 'int64', 'float64', 'float64', 'int64', 'S4', 'S6',
                 'float64', 'float64', 'int64', 'float64', 'float64', 'float64', 'float64',
                     'float64', 'float64', 'S6', 'int64'])

    targets = None

    try:
        db = psycopg2.connect(host='specprod-db.desi.lbl.gov', database='desi', user='desi', password='')
        cursor = db.cursor()

        for i in tqdm(range(5)):

            sga_id = sga_params['SGA_ID'][i]
            ra = sga_params['RA'][i]
            dec = sga_params['DEC'][i]
            major = sga_params['MAJOR'][i]
            ratio = sga_params['RATIO'][i]
            pa = sga_params['PA'][i]
            

            query = f"""SELECT f.targetid, f.tileid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
                        zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
                        zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
                        FROM iron.fiberassign AS f 
                        JOIN iron.zpix AS zp ON f.targetid = zp.targetid 
                        WHERE q3c_ellipse_join(f.target_ra, f.target_dec, {ra}, {dec}, {major}, {ratio}, {pa});"""
            
            if verbose:
                print(query)
    
            cursor.execute(query)
            rows = cursor.fetchall()

            if len(rows)>0:
                targets = Table(list(map(list, zip(*rows))), names=['TARGETID', 'TILEID', 'TARGET_RA', 'TARGET_DEC', 'HEALPIX', 'SURVEY', 'PROGRAM',
                              'Z', 'ZERR', 'ZWARN', 'CHI2', 'DELTACHI2', 'MEAN_FIBER_RA', 'MEAN_FIBER_DEC',
                             'STD_FIBER_RA', 'STD_FIBER_DEC', 'SPECTYPE'],
                dtype=['int64', 'int64', 'float64', 'float64', 'int64', 'S4', 'S6',
                     'float64', 'float64', 'int64', 'float64', 'float64', 'float64', 'float64',
                         'float64', 'float64', 'S6'])
                targets['SGA_ID'] = sga_id
    
                rot_curve_targets = vstack([rot_curve_targets, targets])

    except Exception as error:
        print(error)
    finally:
        if db is not None:
            db.close()

    return rot_curve_targets

#### load tables

In [3]:
pv_tf = Table.read('/global/cfs/cdirs/desi/science/td/pv/desi_pv/savepath_dr9_corr/pv_tf.fits', format='fits')
pv_tf_sga_ids = np.unique(pv_tf['SGA_ID'])
pv_tf[:5]

OBJID,BRICKID,BRICKNAME,RA,DEC,PMRA,PMDEC,REF_EPOCH,OVERRIDE,PVTYPE,PVPRIORITY,POINTINGID,SGA_ID
int64,int32,bytes8,float64,float64,float32,float32,float32,bool,bytes3,int32,int64,int64
262,520796,2267p350,226.62204270638287,34.98227660676318,0.0,0.0,2015.5,True,TFT,1,1,1006008
262,520796,2267p350,226.62323955692426,34.98099178744529,0.0,0.0,2015.5,False,TFT,2,2,1006008
262,520796,2267p350,226.62443636990747,34.97970695638398,0.0,0.0,2015.5,True,TFT,1,3,1006008
2738,514840,2263p337,226.40142777879575,33.68537046528162,0.0,0.0,2015.5,True,TFT,1,1,4331
2738,514840,2263p337,226.40426477759235,33.68450278837041,0.0,0.0,2015.5,False,TFT,2,2,4331


In [4]:
sga = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
sga_dict = {}
for i in range(len(sga)):
    sga_dict[sga['SGA_ID'][i]] = i
sga[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

In [5]:
### create table of sga params
ras = np.zeros(len(pv_tf_sga_ids))
decs = np.zeros(len(pv_tf_sga_ids))
majors = np.zeros(len(pv_tf_sga_ids))
ratios = np.zeros(len(pv_tf_sga_ids))
pas = np.zeros(len(pv_tf_sga_ids))

for i in range(len(pv_tf_sga_ids)):
    sga_id = pv_tf_sga_ids[i]
    idx = sga_dict[sga_id]
    ras[i] = sga['RA'][idx]
    decs[i] = sga['DEC'][idx]
    majors[i] = ((sga['D26'][idx]*0.5*u.arcmin).to(u.degree).value) * 1.01 # convert to deg and add 1 percent dist to get outer edge targs?
    ratios[i] = sga['BA'][idx]
    pas[i] = sga['PA'][idx]

In [6]:
sga_params = Table([pv_tf_sga_ids, ras, decs, majors, ratios, pas],
                   names=['SGA_ID', 'RA', 'DEC', 'MAJOR', 'RATIO', 'PA'],
                   dtype=['int64', 'float64', 'float64', 'float64', 'float64', 'float64'])

In [28]:
sga_params.write('/pscratch/sd/n/nravi/pv_rot_curves/sga_params.fits', format='fits')

### generate iron data table

In [16]:
iron_table = get_rot_curve_targets('iron', sga_params, verbose=0)

connection to server at "specprod-db.desi.lbl.gov" (128.55.212.78), port 5432 failed: FATAL:  the database system is not yet accepting connections
DETAIL:  Consistent recovery state has not been yet reached.



UnboundLocalError: local variable 'db' referenced before assignment

In [44]:
iron_table

TARGETID,TILEID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,PROGRAM,Z,ZERR,ZWARN,CHI2,DELTACHI2,MEAN_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_RA,STD_FIBER_DEC,SPECTYPE,SGA_ID
int64,int64,float64,float64,int64,bytes4,bytes6,float64,float64,int64,float64,float64,float64,float64,float64,float64,bytes6,int64
39627956930219105,21128,202.5446879281934,6.937716745448383,26419,main,bright,0.07405882697719757,5.507764891123595e-06,0,8900.472087532282,4056.44409406418,202.54466847991569,6.937713621590798,0.0,0.0,GALAXY,3


In [8]:
len(sga_params)

129772

### testing temporary table

In [9]:
target_id = iron_table['TARGETID'][0]

db = psycopg2.connect(host='specprod-db.desi.lbl.gov', database='desi', user='desi', password='')
print('connected')
cursor = db.cursor()

cursor.execute("""CREATE TEMP TABLE pv(
                                ra FLOAT,
                                dec FLOAT,
                                major FLOAT,
                                ratio FLOAT,
                                pa FLOAT); 
                                """)
print('temp table created')

for i in range(5):
    cursor.execute(f"""INSERT INTO pv VALUES ({sga_params['RA'][i]}, 
                                                {sga_params['DEC'][i]}, 
                                                {sga_params['MAJOR'][i]}, 
                                                {sga_params['RATIO'][i]},
                                                {sga_params['PA'][i]})""")

print('sga params inserted in temp table')

print('pv table')

cursor.execute("""SELECT * FROM pv""")
rows = cursor.fetchall()
print(rows)

print('querying database w SGA params')
# cursor.execute(f"""SELECT f.targetid, f.tileid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
#                         zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
#                         zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
#                         FROM pv as pv, iron.fiberassign AS f
#                         JOIN iron.zpix AS zp ON f.targetid = zp.targetid
#                         WHERE q3c_ellipse_join(f.target_ra, f.target_dec, pv.ra, pv.dec, pv.major, pv.ratio, pv.pa);""")

START = datetime.datetime.now()


cursor.execute(f"""SELECT f.targetid, f.tileid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
                        zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
                        zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
                        FROM pv, iron.fiberassign AS f
                        JOIN iron.zpix AS zp ON f.targetid = zp.targetid
                        WHERE q3c_ellipse_join(f.target_ra, f.target_dec, pv.ra, pv.dec, pv.major, pv.ratio, pv.pa);""")

print('Runtime: ', datetime.datetime.now() - START)

rows = cursor.fetchall()


NameError: name 'iron_table' is not defined

#### create index for temp table

In [7]:
# version with indexing table
target_id = iron_table['TARGETID'][0]

db = psycopg2.connect(host='specprod-db.desi.lbl.gov', database='desi', user='desi', password='')
print('connected')
cursor = db.cursor()

cursor.execute("""CREATE TEMP TABLE pv(
                                ra FLOAT,
                                dec FLOAT,
                                major FLOAT,
                                ratio FLOAT,
                                pa FLOAT); 
                                """)


for i in range(5):
    cursor.execute(f"""INSERT INTO pv VALUES ({sga_params['RA'][i]}, 
                                                {sga_params['DEC'][i]}, 
                                                {sga_params['MAJOR'][i]}, 
                                                {sga_params['RATIO'][i]},
                                                {sga_params['PA'][i]})""")

print('sga params inserted in temp table')

cursor.execute("""CREATE INDEX ON pv (q3c_ang2ipix(ra, dec));""")
cursor.execute("""ANALYZE pv;""")

print('created index')

print('pv table')

cursor.execute("""SELECT * FROM pv""")
rows = cursor.fetchall()
print(rows)

print('querying database w SGA params')

START = datetime.datetime.now()

cursor.execute(f"""SELECT f.targetid, f.tileid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
                        zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
                        zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
                        FROM pv, iron.fiberassign AS f
                        JOIN iron.zpix AS zp ON f.targetid = zp.targetid
                        WHERE q3c_ellipse_join(f.target_ra, f.target_dec, pv.ra, pv.dec, pv.major, pv.ratio, pv.pa);""")

print('Runtime: ', datetime.datetime.now() - START)

rows = cursor.fetchall()



NameError: name 'iron_table' is not defined

In [35]:
len(rows)

1

#### trying to insert all sga params into temp table

In [60]:
tuples = [tuple(x) for x in sga_params['RA', 'DEC', 'MAJOR', 'RATIO', 'PA']]

In [63]:
db = psycopg2.connect(host='specprod-db.desi.lbl.gov', database='desi', user='desi', password='')
print('connected')
cursor = db.cursor()

START = datetime.datetime.now()

cursor.execute("""CREATE TEMP TABLE pv(
                                ra FLOAT,
                                dec FLOAT,
                                major FLOAT,
                                ratio FLOAT,
                                pa FLOAT); 
                                """)


execute_values(cursor, """INSERT INTO pv (ra, dec, major, ratio, pa) VALUES %s """, tuples)

print('Runtime : ', datetime.datetime.now() - START)

cursor.execute("""SELECT * FROM pv""")
rows = cursor.fetchall()

connected
Runtime :  0:00:03.056472


In [30]:
# check that temp table was indeed destroyed -- SUCCESS
db = psycopg2.connect(host='specprod-db.desi.lbl.gov', database='desi', user='desi', password='')
print('connected')
cursor = db.cursor()
cursor.execute(f"""INSERT INTO pv VALUES ({target_id})""")


connected


UndefinedTable: relation "pv" does not exist
LINE 1: INSERT INTO pv VALUES (39627956930219105)
                    ^


In [8]:
def adapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

In [9]:
def get_rot_curve_targets_temp_table(redux, sga_params, verbose=False):
    """Get rotation curve targets from the DESI production DB for a given spectroscopic reduction.
    
    Parameters
    ----------
    redux : str
        Spectroscopic reduction. can be 'fuji', 'guadalupe', 'iron', or 'loa'

    sga_params: list of tuples
        list of tuples of the form [(sga_id, ra, dec, major, pa)]

        sga_id : int
            SGA ID from pv table
        
        ra : float
            galaxy RA from SGA [deg]
        dec : float
            galaxy dec from SGA [deg]
        major : float
            semi-major axis (D26/2) from SGA [deg]
        ratio : float
            axis ratio (BA) from SGA
        pa : float
            position angle east-of-north from SGA [deg]
    
    Returns
    -------
    targets : Table
        Table of observations.
    """
    


    targets = None

    try:
        print(1)
        register_adapter(np.int64, adapt_numpy_int64)
        db = psycopg2.connect(host='specprod-db.desi.lbl.gov', database='desi', user='desi', password='')
        cursor = db.cursor()

        # create temporary pv table

        cursor.execute("""CREATE TEMP TABLE pv(
                                sgaid BIGINT,
                                ra FLOAT,
                                dec FLOAT,
                                major FLOAT,
                                ratio FLOAT,
                                pa FLOAT); 
                                """)
        print('temp table created')
        # put sga parameters into the temporary table
        execute_values(cursor, """INSERT INTO pv(sgaid, ra, dec, major, ratio, pa) VALUES %s """, tuples)

        print('temp table filled')
        # create index for pv table
        
        cursor.execute("""CREATE INDEX ON pv (q3c_ang2ipix(ra, dec));""")
        cursor.execute("""ANALYZE pv;""")

        print('temp table indexed')    
        # get all targets within sga ellipses
        

        # query = f"""SELECT f.targetid, f.tileid, pv.sgaid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
        #                 zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
        #                 zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
        #                 FROM pv, iron.fiberassign AS f
        #                 JOIN iron.zpix AS zp ON f.targetid = zp.targetid
        #                 WHERE q3c_ellipse_join(f.target_ra, f.target_dec, pv.ra, pv.dec, pv.major, pv.ratio, pv.pa);"""

        

        query = f"""SELECT f.targetid, f.tileid, pv.sgaid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
                        zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
                        zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
                        FROM pv, iron.fiberassign AS f
                        JOIN iron.zpix AS zp ON f.targetid = zp.targetid
                        WHERE f.target_ra < (pv.ra + pv.major) AND f.target_ra > (pv.ra - pv.major) AND f.target_dec < (pv.dec + pv.major) AND f.target_dec > (pv.dec - pv.major);"""
        
        if verbose:
            print(query)

        cursor.execute(query)
        rows = cursor.fetchall()

        if len(rows)>0:
            targets = Table(list(map(list, zip(*rows))), names=['TARGETID', 'TILEID', 'SGA_ID', 'TARGET_RA', 
                                                                'TARGET_DEC', 'HEALPIX', 'SURVEY', 'PROGRAM',
                          'Z', 'ZERR', 'ZWARN', 'CHI2', 'DELTACHI2', 'MEAN_FIBER_RA', 'MEAN_FIBER_DEC',
                         'STD_FIBER_RA', 'STD_FIBER_DEC', 'SPECTYPE'],
            dtype=['int64', 'int64', 'int64', 'float64', 'float64', 'int64', 'S4', 'S6',
                 'float64', 'float64', 'int64', 'float64', 'float64', 'float64', 'float64',
                     'float64', 'float64', 'S6'])


    except Exception as error:
        print(error)
    finally:
        if db is not None:
            db.close()
    
    return targets

In [ ]:
# test query on small table

tuples = [tuple(x) for x in sga_params[:20]]
START = datetime.datetime.now()
iron_test = get_rot_curve_targets_temp_table('iron', tuples, verbose=True)
print('done -- ', datetime.datetime.now() - START)

1
temp table created
temp table filled
temp table indexed
SELECT f.targetid, f.tileid, pv.sgaid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
                        zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
                        zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
                        FROM pv, iron.fiberassign AS f
                        JOIN iron.zpix AS zp ON f.targetid = zp.targetid
                        WHERE q3c_ellipse_join(f.target_ra, f.target_dec, pv.ra, pv.dec, pv.major, pv.ratio, pv.pa);


In [23]:
iron_test

TARGETID,TILEID,SGA_ID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,PROGRAM,Z,ZERR,ZWARN,CHI2,DELTACHI2,MEAN_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_RA,STD_FIBER_DEC,SPECTYPE
int64,int64,int64,float64,float64,int64,bytes4,bytes6,float64,float64,int64,float64,float64,float64,float64,float64,float64,bytes6
2389570544992262,22682,55,199.7311685666833,1.7247142071276316,26207,main,bright,0.04422788661807545,4.938238651323468e-06,0,7728.932424351573,987.540528729558,199.73116845226696,1.724758061816374,0.0,0.0,GALAXY
2389570544992261,24243,55,199.73007238519418,1.7278318158265755,26207,main,bright,0.043724082715951046,2.0002041200493262e-05,0,7557.245729940012,87.38531109504402,199.7300746094984,1.7278269937409718,0.036127813,0.024903476,GALAXY
2389570544992261,25837,55,199.73007238519418,1.7278318158265755,26207,main,bright,0.043724082715951046,2.0002041200493262e-05,0,7557.245729940012,87.38531109504402,199.7300746094984,1.7278269937409718,0.036127813,0.024903476,GALAXY
39627853494489271,23090,117,138.9354018819327,2.69476220880469,27323,main,bright,0.09003983347898953,2.775517735062233e-05,0,8013.983035285026,896.4243823117577,138.93543745285615,2.694729924956867,0.0,0.0,GALAXY
39628040673694224,11467,74,208.88083454985727,10.457524716375113,26481,main,dark,2.5355102189406797,0.00036370605063924973,0,7893.974747925997,547.2669170121662,208.88083079857302,10.457523811578822,0.0,0.0,QSO
39628040673694353,20123,74,208.88595452038825,10.45038023846306,26481,main,bright,0.03812963730157509,3.612668794954218e-06,0,7591.736131072044,1762.430552124977,208.88589635304947,10.450355697007257,0.0,0.0,GALAXY
39627956930219105,21128,3,202.5446879281934,6.937716745448383,26419,main,bright,0.07405882697719757,5.507764891123595e-06,0,8900.472087532282,4056.44409406418,202.54466847991569,6.937713621590798,0.0,0.0,GALAXY
2390410135601155,22720,38,211.66381961968403,39.527428957913536,10641,main,bright,0.08058017670098534,8.151790687169398e-06,0,7811.470571607351,355.43456426262856,211.66383615121413,39.52744095540257,0.0,0.0,GALAXY
39628256852317379,23338,63,234.874346595437,19.700467453856458,8599,main,bright,0.04568205634248944,2.8071279188846497e-06,0,7882.064212948084,4443.821897834539,234.87436146772416,19.70047756527694,0.0,0.0,GALAXY


In [10]:
tuples = [tuple(x) for x in sga_params[:20]]
START = datetime.datetime.now()
iron_test2 = get_rot_curve_targets_temp_table('iron', tuples, verbose=True)
print('done -- ', datetime.datetime.now() - START)

1
temp table created
temp table filled
temp table indexed
SELECT f.targetid, f.tileid, pv.sgaid, f.target_ra, f.target_dec, zp.healpix, zp.survey, zp.program,
                        zp.z, zp.zerr, zp.zwarn, zp.chi2, zp.deltachi2, zp.mean_fiber_ra, zp.mean_fiber_dec,
                        zp.std_fiber_ra, zp.std_fiber_dec, zp.spectype
                        FROM pv, iron.fiberassign AS f
                        JOIN iron.zpix AS zp ON f.targetid = zp.targetid
                        WHERE f.target_ra < (pv.ra + pv.major) AND f.target_ra > (pv.ra - pv.major) AND f.target_dec < (pv.dec + pv.major) AND f.target_dec > (pv.dec - pv.major);
done --  0:00:52.520976


In [11]:
iron_test2

TARGETID,TILEID,SGA_ID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,PROGRAM,Z,ZERR,ZWARN,CHI2,DELTACHI2,MEAN_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_RA,STD_FIBER_DEC,SPECTYPE
int64,int64,int64,float64,float64,int64,bytes4,bytes6,float64,float64,int64,float64,float64,float64,float64,float64,float64,bytes6
2389570544992262,22682,55,199.7311685666833,1.7247142071276316,26207,main,bright,0.04422788661807545,4.938238651323468e-06,0,7728.932424351573,987.540528729558,199.73116845226696,1.724758061816374,0.0,0.0,GALAXY
2389570544992261,24243,55,199.73007238519418,1.7278318158265755,26207,main,bright,0.043724082715951046,2.0002041200493262e-05,0,7557.245729940012,87.38531109504402,199.7300746094984,1.7278269937409718,0.036127813,0.024903476,GALAXY
2389570544992261,25837,55,199.73007238519418,1.7278318158265755,26207,main,bright,0.043724082715951046,2.0002041200493262e-05,0,7557.245729940012,87.38531109504402,199.7300746094984,1.7278269937409718,0.036127813,0.024903476,GALAXY
39627853494489271,23090,117,138.9354018819327,2.69476220880469,27323,main,bright,0.09003983347898953,2.775517735062233e-05,0,8013.983035285026,896.4243823117577,138.93543745285615,2.694729924956867,0.0,0.0,GALAXY
39628040673694224,11467,74,208.88083454985727,10.457524716375113,26481,main,dark,2.5355102189406797,0.00036370605063924973,0,7893.974747925997,547.2669170121662,208.88083079857302,10.457523811578822,0.0,0.0,QSO
39628040673694353,20123,74,208.88595452038825,10.45038023846306,26481,main,bright,0.03812963730157509,3.612668794954218e-06,0,7591.736131072044,1762.430552124977,208.88589635304947,10.450355697007257,0.0,0.0,GALAXY
39627956930219105,21128,3,202.5446879281934,6.937716745448383,26419,main,bright,0.07405882697719757,5.507764891123595e-06,0,8900.472087532282,4056.44409406418,202.54466847991569,6.937713621590798,0.0,0.0,GALAXY
2390410135601155,22720,38,211.66381961968403,39.527428957913536,10641,main,bright,0.08058017670098534,8.151790687169398e-06,0,7811.470571607351,355.43456426262856,211.66383615121413,39.52744095540257,0.0,0.0,GALAXY
39628256852317379,23338,63,234.874346595437,19.700467453856458,8599,main,bright,0.04568205634248944,2.8071279188846497e-06,0,7882.064212948084,4443.821897834539,234.87436146772416,19.70047756527694,0.0,0.0,GALAXY


In [20]:
# targ_list = tf_mastertable[tf_mastertable['SGA_ID'] == sga_id]
for sga_id in np.unique(iron_test2['SGA_ID']):

    sga_row = sga[sga_dict[sga_id]]

    ra, dec = sga_row['RA', 'DEC']
    
    # D26 in arcmin
    d26 = sga_row['D26']
    z = sga_row['Z_LEDA']
    
    npix = np.minimum(int(2 * d26*60/0.262), 512)
    # npix = int(2 * d26*60/0.262)
    
    cache_dir = '/pscratch/sd/n/nravi/pv_rot_curves/cache/'
    
    img_file, wcs = get_cutout(sga_id, ra, dec, size=npix, cache_dir=cache_dir,verbose=True)
    img = mpl.image.imread(img_file)
    
    fig1 = plt.figure(figsize=(5,5))
    
    ax = fig1.add_subplot(111, projection=wcs)
    ax.imshow(np.flip(img, axis=0))
    ax.set(xlabel='ra', ylabel='dec')
    ax.text(int(0.02*npix), int(0.85*npix), 'SGA_ID: {}\n$z={{{:.4f}}}$'.format(sga_id, z), fontsize=9, color='yellow')
    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted');
    
    # Add the location of the DESI fibers.
    # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
    r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='black', facecolor='none', alpha=0.8, lw=3,
                         transform=ax.get_transform('icrs'))
    r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='red', facecolor='none', alpha=0.8, lw=2,
                         transform=ax.get_transform('icrs'))
    ax.add_patch(r1)
    ax.add_patch(r2)

    targs = iron_test2[iron_test2['SGA_ID'] == sga_id]
    
    for targ in targs:
        ra, dec = targ['TARGET_RA'], targ['TARGET_DEC']
        
        edgecolor2 = 'orange'
    
        # Add the location of the DESI fibers.
        # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
        r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor='lightcoral', facecolor='none', alpha=1, lw=3,
                             transform=ax.get_transform('icrs'))
        r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor=edgecolor2, facecolor='none', alpha=0.8, lw=2,
                             transform=ax.get_transform('icrs'))
        ax.add_patch(r1)
        ax.add_patch(r2)
        
        ax.text(ra, dec, str(targ['TARGETID']), transform=ax.get_transform('icrs'), color='green', fontsize=6)
    
    # fig1.subplots_adjust(top=0.85, right=0.85, bottom=0.15, left=0.15)
    fig1.savefig(cache_dir + str(sga_id) + '.jpg')
    fig1.clear()

/pscratch/sd/n/nravi/pv_rot_curves/cache/2.jpg exists.
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=202.5444619671207&dec=6.9346244322326624&zoom=14&layer=ls-dr9&size=253&sga
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=211.66203166014844&dec=39.52724032794356&zoom=14&layer=ls-dr9&size=191&sga
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=255.6410175635765&dec=36.68179095091098&zoom=14&layer=ls-dr9&size=174&sga
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=199.73062047638814&dec=1.7262730115560398&zoom=14&layer=ls-dr9&size=227&sga
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=234.87534259414386&dec=19.697185738728333&zoom=14&layer=ls-dr9&size=297&sga
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=209.15740662967733&dec=36.2461329967411&zoom=14&layer=ls-dr9&size=396&sga
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=208.88595452038825&dec=10.45038023846306&zoom=14&layer=ls-dr9&size=429&sga
Get https://www.legacysurvey.org/viewer/cu

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

...

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

In [12]:
def get_cutout(SGA_ID, ra, dec, size, cache_dir, verbose=False):
    '''Grab and cache legacy survey cutouts.
    
    Parameters
    ----------
    SGA_ID : int
        SGA ID for galaxy.
    ra : float
        Right ascension (degrees).
    dec : float
        Declination (degrees).
    size : float
        size of cutout.
    cache_dir : string
        cache location
    verbose : bool
        Add some status messages if true.
        
    Returns
    -------
    img_name : str
        Name of JPG cutout file written after query.
    w : astropy.wcs.WCS
        World coordinate system for the image.
    '''
    
    # Either load an existing image or download a cutout.
    img_name = cache_dir + '{}.jpg'.format(SGA_ID)
    
    
    if os.path.exists(img_name):
        if verbose:
            print('{} exists.'.format(img_name))

    else:
        img_url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&zoom=14&layer=ls-dr9&size={}&sga'.format(ra, dec, size)
        if verbose:
            print('Get {}'.format(img_url))
            
        with open(img_name, 'wb') as handle: 
            response = requests.get(img_url, stream=True) 
            if not response.ok: 
                print(response) 
            for block in response.iter_content(1024): 
                if not block: 
                    break 
                handle.write(block)
                
    # Set up the WCS.
    wcs_input_dict = {
        'CTYPE1': 'RA---TAN',
        'CUNIT1': 'deg',
        'CDELT1': -0.262/3600,
        'CRPIX1': size/2 + 0.5,
        'CRVAL1': ra,
        'NAXIS1': size,
        'CTYPE2': 'DEC--TAN',
        'CUNIT2': 'deg',
        'CDELT2': 0.262/3600,
        'CRPIX2': size/2 + 0.5,
        'CRVAL2': dec,
        'NAXIS2': size
    }
    w = WCS(wcs_input_dict)
    
    return img_name, w

In [32]:
(52/20)*len(sga_params) /60 /60 

93.72422222222222

In [33]:
len(sga_params)

129772